In [1]:
import cv2
from matplotlib import pyplot as plt
import imutils
import numpy as np
import easyocr
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

/Users/samettenha/miniforge3/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: dlopen(/Users/samettenha/miniforge3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/samettenha/miniforge3/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/Users/malfet/miniforge3/envs/py_39_torch-1.10.2/lib/libpng16.16.dylib' (no such file), '/Users/malfet/miniforge3/envs/py_39_torch-1.10.2/lib/libpng16.16.dylib' (no such file), '/Users/malfet/miniforge3/envs/py_39_torch-1.10.2/lib/libpng16.16.dylib' (no such file), '/Users/malfet/miniforge3/envs/py_39_torch-1.10.2/lib/libpng16.16.dylib' (no such file), '/Users/samettenha/miniforge3/lib/python3.9/lib-dynload/../../libpng16.16.dylib' (no such file), '/Users/samettenha/miniforge3/bin/../lib/libpng16.16.dylib' (no such file), '/usr/local/lib/libpng16.16.dylib' (no such file), '/usr/lib/libpng16.16.dylib' 

In [4]:
# add check_out Time
def anpr_v4():
    check_in = input('\n\nPlease press A to open the gate and record your Plate Info:').lower()
    #
    # check_in 
    if check_in == 'a':
        #
        # record time say welcome
        time_in = datetime.now()#.strftime("%Y-%m-%d %H:%M:%S")
        print('Welcome you check in at:', time_in.strftime("%Y-%m-%d %H:%M:%S"))
        
        #
        # take a picture an record to localize the coordinates of the plate
        gray = cv2.cvtColor(cv2.imread('01_picture/image_02.jpeg'), cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(5,5),0) 
        canny = cv2.Canny(blur, threshold1=30,threshold2=200)
        
        keypoints = cv2.findContours(canny.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
        contours = sorted(imutils.grab_contours(keypoints), key=cv2.contourArea, reverse=True)[:10]
        
        location = []
        for contour in contours:
            approx = cv2.approxPolyDP(contour, 10, True) # approxPolyDP(input_curve, epsilon, closed) -> https://www.programcreek.com/python/example/89328/cv2.approxPolyDP
            if len(approx) == 4:
                location = approx
                break
                
        mask = np.zeros(gray.shape, dtype=np.uint8)

        cv2.drawContours(mask, [location], 0,255, -1)

        (x,y) = np.where(mask==255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1:x2, y1:y2]
        
        # read out the plate info
        result = easyocr.Reader(['en']).readtext(cropped_image)
        
        text_sum=[]
        for x in range(len(result)):
            text_sum.append(result[x][-2])    
        plate = ' '.join(text_sum)
        
        print('We record your Plate as:', plate)
        
        #
        # Check out
        check_out = input('\n\nPlease type in your plate number to get ticket bill:').lower()
        
        if check_out == plate.lower():
            time_out = datetime.now()
            print('You check out at:', time_out.strftime("%Y-%m-%d %H:%M:%S"))
            visit_time = int((time_out - time_in).total_seconds())
            payment = visit_time * 0.09
            print('Your bill amounts to',round(payment,2),'€')
        else:
            print('bye bye')
        
        
    else:
        print('bye bye')

In [6]:
anpr_v4()



Please press A to open the gate and record your Plate Info:a


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Welcome you check in at: 2022-03-11 12:40:42
We record your Plate as: ED HV 919


Please type in your plate number to get ticket bill:ed hv 919
You check out at: 2022-03-11 12:40:54
Your bill amounts to 1.08 €
